# Sequence to Sequence with attention

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
* [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)
* [Neural Machine Translation with Attention from Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb)

In [8]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os
print(tf.__version__)

2.10.0


In [9]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]

In [15]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}

print(sum(sources,[]))
print(s_vocab)
pprint(source2idx)

['I', 'feel', 'hungry', 'tensorflow', 'is', 'very', 'difficult', 'tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning', 'tensorflow', 'is', 'very', 'fast', 'changing']
['<pad>', 'I', 'a', 'changing', 'deep', 'difficult', 'fast', 'feel', 'for', 'framework', 'hungry', 'is', 'learning', 'tensorflow', 'very']
{'<pad>': 0,
 'I': 1,
 'a': 2,
 'changing': 3,
 'deep': 4,
 'difficult': 5,
 'fast': 6,
 'feel': 7,
 'for': 8,
 'framework': 9,
 'hungry': 10,
 'is': 11,
 'learning': 12,
 'tensorflow': 13,
 'very': 14}


In [4]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}

pprint(target2idx)

{'<bos>': 1,
 '<eos>': 2,
 '<pad>': 0,
 '고프다': 3,
 '나는': 4,
 '딥러닝을': 5,
 '매우': 6,
 '배가': 7,
 '변화한다': 8,
 '빠르게': 9,
 '어렵다': 10,
 '위한': 11,
 '텐서플로우는': 12,
 '프레임워크이다': 13}


In [5]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    """
     시퀀스 데이터를 전처리하는 함수
    
     Parameters:
     - sequences: 입력 시퀀스 데이터
     - max_len: 패딩 후의 최대 길이
     - dic: 토큰을 정수로 매핑하는 딕셔너리
     - mode: 'source' 또는 'target' 중 하나 선택
    
     Returns:
     - 'source' 모드일 경우:
         - s_len: 각 문장의 길이를 담은 리스트
         - s_input: 패딩된 인코더 입력 시퀀스
    
     - 'target' 모드일 경우:
         - t_len: 각 디코더 입력 시퀀스의 길이를 담은 리스트
         - t_input: 패딩된 디코더 입력 시퀀스
         - t_output: 패딩된 디코더 출력 시퀀스
     """
    if mode == 'source':
        #  # Source 모드 전처리 (인코더 입력)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        # 각 문장의 길이를 계산하여 리스트로 저장
        # map(함수,인자) 따라서 함수를 sequences에 적용해서 중복업는 인덱스를가져옴 dic.get은 인덱스를 가져오는 함수
        s_len = list(map(lambda sentence : len(sentence), s_input))
        # 패딩 적용
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        # truncating 정해진 길이보다길면 뒤를 자른다
        return s_len, s_input

    elif mode == 'target':
        # Target 모드 전처리 (디코더 입력 및 출력)
        # 디코더 입력에는 '<bos>' (문장의 시작)를 추가하고, '<eos>' (문장의 끝)를 추가한 후에 토큰을 딕셔너리에서 매핑된 정수로 변환

        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        # 각 디코더 입력 시퀀스의 길이를 계산하여 리스트로 저장    
        t_len = list(map(lambda sentence : len(sentence), t_input))
        # 패딩 적용
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')

        # 디코더 출력에는 '<eos>'를 추가한 후에 토큰을 딕셔너리에서 매핑된 정수로 변환
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')

        return t_len, t_input, t_output

In [6]:
# preprocessing for source
s_max_len = 10
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[3, 4, 7, 5] [[ 1  7 10  0  0  0  0  0  0  0]
 [13 11 14  5  0  0  0  0  0  0]
 [13 11  2  9  8  4 12  0  0  0]
 [13 11 14  6  3  0  0  0  0  0]]


In [7]:
# preprocessing for target
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 5, 6, 6] [[ 1  4  7  3  2  0  0  0  0  0  0  0]
 [ 1 12  6 10  2  0  0  0  0  0  0  0]
 [ 1 12  5 11 13  2  0  0  0  0  0  0]
 [ 1 12  6  9  8  2  0  0  0  0  0  0]] [[ 4  7  3  2  0  0  0  0  0  0  0  0]
 [12  6 10  2  0  0  0  0  0  0  0  0]
 [12  5 11 13  2  0  0  0  0  0  0  0]
 [12  6  9  8  2  0  0  0  0  0  0  0]]


# hyper-param

In [ ]:
# hyper-parameters
epochs = 100
batch_size = 4
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 128

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [ ]:
def gru(units):
    return tf.keras.layers.GRU(units,
                               return_sequences=True,
                               return_state=True,
                               recurrent_activation='sigmoid',
                               recurrent_initializer='glorot_uniform')

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        """
        Encoder 클래스 초기화

        Parameters:
        - vocab_size: 어휘 사전 크기
        - embedding_dim: 임베딩 차원
        - enc_units: GRU 유닛 수
        - batch_sz: 배치 크기
        """
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)

    def call(self, x, hidden):
        """
        Encoder의 호출 메서드

        Parameters:
        - x: 입력 시퀀스
        - hidden: 초기 은닉 상태

        Returns:
        - output: GRU의 출력
        - state: GRU의 상태
        output: 각 시간 단계(time step)에서의 GRU 레이어의 출력을 나타냅니다. 시퀀스에 대한 정보
        state: GRU 레이어의 최종 상태. 시퀀스 전체를 처리한 후의 은닉 상태입니다. 이는 다음 시간 단계의 초기 은닉 상태로 사용됩니다.
        """
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        # print("state: {}".format(state.shape))
        # print("output: {}".format(state.shape))

        return output, state

    def initialize_hidden_state(self):
        """
        초기 은닉 상태를 0으로 초기화하는 메서드

        Returns:
        - 초기화된 은닉 상태
        """
        return tf.zeros((self.batch_sz, self.enc_units))


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        """
        Decoder 클래스 초기화

        Parameters:
        - vocab_size: 어휘 사전 크기
        - embedding_dim: 임베딩 차원
        - dec_units: GRU 유닛 수
        - batch_sz: 배치 크기
        """
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)

        # 어텐션을 위한 가중치 설정
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):
        """
        Decoder의 호출 메서드

        Parameters:
        - x: 입력 시퀀스
        - hidden: 초기 은닉 상태
        - enc_output: 인코더의 출력

        Returns:
        - x: 디코더 출력
        - state: 디코더의 상태
        - attention_weights: 어텐션 가중치
        """
        # enc_output shape == (batch_size, max_length, hidden_size)

        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # attention score 계산을 위해 차원 추가
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # hidden을 확장해서 shape를 맞추기 위함

        # score 계산: score = FC(tanh(FC(EO) + FC(H)))
        # score shape == (batch_size, max_length, 1)
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        # 아웃풋과 히든을 Dense로 묶어서 스코어를 계산
        # attention weights 계산: softmax(score, axis=1)
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        # 가중치를 계산
        # context_vector 계산: context_vector = sum(attention weights * EO, axis=1)
        # context_vector shape == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        # 결과는 element-wise 곱셈이 적용된다.
        # [[[0.2, 0.4, 0.6], [1.2, 1.5, 1.8], [0.7, 0.8, 0.9], [4.0, 4.4, 4.8]],  # 첫 번째 문장의 context_vector
        #  [[1.3, 1.4, 1.5], [6.4, 6.8, 7.2], [3.8, 4.0, 4.2], [6.6, 6.9, 7.2]]]  # 두 번째 문장의 context_vector

        # 여기서 각각의 가중치가 적용됨.
        context_vector = tf.reduce_sum(context_vector, axis=1)
        # 결과는 axis=1을 따라 각 문장에 대한 합이 계산된다.
        # [[6.1, 7.1, 8.1],  # 첫 번째 문장의 context_vector
        #  [18.1, 18.1, 18.6]]  # 두 번째 문장의 context_vector

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # concatenated vector를 GRU에 전달
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)

        return x, state, attention_weights

    def initialize_hidden_state(self):
        """
        초기 은닉 상태를 0으로 초기화하는 메서드

        Returns:
        - 초기화된 은닉 상태
        """
        return tf.zeros((self.batch_sz, self.dec_units))


In [ ]:
# Encoder와 Decoder 인스턴스 생성
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

# 손실 함수 정의
def loss_function(real, pred):
    """
    손실 함수 정의

    Parameters:
    - real: 실제 값
    - pred: 예측 값

    Returns:
    - 손실 값
    """
    # 패딩된 부분에 대한 마스크 생성
    mask = 1 - np.equal(real, 0)
    # 0인 부분은 1이 되므로 패딩된 부분은 손실에 영향을 끼치지 않게 0인부분은 1로 만듬
    # sparse_softmax_cross_entropy_with_logits를 이용한 손실 계산
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

# 옵티마이저 생성
optimizer = tf.keras.optimizers.Adam()

# 체크포인트 생성 (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

# TensorBoard를 위한 로그 디렉토리 생성 및 작성기(writer) 생성
summary_writer = tf.summary.create_file_writer(logdir=checkpoint_dir)


In [ ]:
for epoch in range(epochs):
    # 전체데이타에 대해
    # 초기 은닉 상태 초기화
    hidden = encoder.initialize_hidden_state()
    total_loss = 0

    # 각미니배치에 대해 - 데이타를 불러옴
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        # 그래디언트 계산을 위한 테이프 생성
        with tf.GradientTape() as tape:
            # 인코더에 소스 문장을 전달하여 출력 및 은닉 상태 획득
            enc_output, enc_hidden = encoder(s_input, hidden)

            # 디코더의 초기 은닉 상태 설정
            dec_hidden = enc_hidden

            # 디코더의 입력으로 시작 토큰('<bos>')을 추가
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #[target2idx['<bos>']] * batch_size: <bos>(시작 토큰)에 해당하는 인덱스를 batch_size만큼 복제한 리스트를 생성합니다.
            # 따라서 이 부분은 [<bos>, <bos>, ..., <bos>]와 같은 형태가 됩니다
            #리스트를 1차원에서 2차원으로 차원을 확장합니다. 이렇게 하면 모양이 (batch_size, 1)인 텐서가 됩니다.

             #Teacher Forcing 방법을 사용하여 디코더를 학습
            for t in range(1, t_input.shape[1]): # t_input.shape[1]은 문장의 길이 (단어수)에서 -1만큼 첫부분만 뺌
                # 디코더에 현재 입력과 은닉 상태 전달하여 예측값 획득
                # 예측값, 어텐션 가중치
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                # 손실 계산
                loss += loss_function(t_input[:, t], predictions)

                # 다음 입력으로는 현재 타임 스텝의 타겟 사용 (Teacher Forcing)
                dec_input = tf.expand_dims(t_input[:, t], 1)

        # 배치 손실 계산
        batch_loss = (loss / int(t_input.shape[1]))

        # 전체 손실 누적
        total_loss += batch_loss

        # 모든 모델 변수에 대한 그래디언트 계산
        variables = encoder.variables + decoder.variables
        gradient = tape.gradient(loss, variables)

        # 그래디언트 업데이트
        optimizer.apply_gradients(zip(gradient, variables))

    # 일정 주기로 모델 저장 및 현재 손실 출력
    if epoch % 10 == 0:
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                                              total_loss / n_batch,
                                                              batch_loss.numpy()))
        checkpoint.save(file_prefix=checkpoint_prefix)


In [ ]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

#     sentence = preprocess_sentence(sentence)

    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# result, sentence, attention_plot = evaluate(sentence, encoder, decoder, source2idx, target2idx,
#                                             s_max_len, t_max_len)

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
sentence = 'I feel hungry'
# sentence = 'tensorflow is a framework for deep learning'

translate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)